In [ ]:
import pandas as pd
import numpy as np

In [ ]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
# grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist']).count().reset_index()
# namesOnly = grammyAwards.groupby(['Name', 'Artist'])
# grammyAwards = namesOnly['Genre'].agg(lambda x: "".join(x)).reset_index(name="Genre")
grammyAwards.head()